# Joke And Critique

This notebook exercises multiple aspects of experimenting with basic LLM prompting

 - Initialization and use of python logging to keep dev logs separate from your jupyter cell output
   - Collecting OpenAI low level logs if needed
   - Generating app-level logs
 - Basic OpenAI ChatEndpoint API calls
   - Turn 1: create a joke
   - Turn 2: Critique that joke
 - Gradio UI with an input widget and two customized textarea widgets

In [ ]:
import os

# If you want to log OpenAI's python library itself, also set the log level for this
# normally, limit this to warning/error and keep your own logging at debug levels.
# If this doesn't work right away, restart the kernel after changing the log-level
os.environ["OPENAI_LOG"]="debug"

# Setup logging 
# Note that module needs to be reloaded for our config to take as Jupyter already configures it
# which makes all future configs no-ops.
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', 
                    level=logging.DEBUG, 
                    datefmt='%I:%M:%S')

## Configure for colab

In [2]:
import os

# You could do one of the two.
# Either paste your OpenAI Key here or put it in secrets
if 'google.colab' in str(get_ipython()):
  from google.colab import userdata
  logging.debug("Tryign to fetch OPENAI_API_KEY from your secrets. Remember to make it available to this notebook")
  os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

logging.debug("Checking if OPENAI_API_KEY is available")
assert(os.environ.get("OPENAI_API_KEY"))

01:55:20 DEBUG:Checking if OPENAI_API_KEY is available


In [3]:
# Lots has changed recently
# See https://github.com/openai/openai-python/discussions/742 for migration to the new API
import openai

openai.api_key = os.environ.get("OPENAI_API_KEY")

def get_completion(prompt, model="gpt-4o-mini", temperature=0) -> str:
    messages = [{"role":"user", "content":prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature)
    return response.choices[0].message.content

In [ ]:
import logging

# Use this to explore available loggers
# If there is a logger and you are not provided an env-var top control log level, 
# you can directly call logger.setLevel(Logging.DEBUG) to collect logs.
def get_available_loggers():
    return [logging.getLogger(name) for name in logging.root.manager.loggerDict]

In [4]:
def joke_and_critique(topic:str) -> str:
    logging.debug("Starting joke_and_critique")
    joke = get_completion(
        f"""Write your best joke about the following: {topic}"""
    )
    logging.debug("Done with Joke Generation")

    critique = get_completion(
        f"Give a thorough analysis and critique of the following joke: {joke}")
    logging.debug("Done with Critique Generation")

    return joke, critique

In [5]:
# This is somehow completely broken on recent mamba release (3.12.5 and 3.13.2)
# start with a new penv and use pip
import gradio as gr

ui = gr.Interface(fn=joke_and_critique, 
                  inputs=gr.Textbox(lines=1, placeholder="Joke Topic", label="Joke Topic"),
                  outputs=[
                      gr.Textbox(lines=3, placeholder="Joke", label="Joke"),
                      gr.Textbox(lines=6, placeholder="Joke Critique", label="Critique"),
                  ])
ui.launch()

06:17:56 DEBUG:connect_tcp.started host='api.gradio.app' port=443 local_address=None timeout=3 socket_options=None
06:17:56 DEBUG:Importing BlpImagePlugin
06:17:56 DEBUG:Importing BmpImagePlugin
06:17:56 DEBUG:Importing BufrStubImagePlugin
06:17:56 DEBUG:Importing CurImagePlugin
06:17:56 DEBUG:Importing DcxImagePlugin
06:17:56 DEBUG:Importing DdsImagePlugin
06:17:56 DEBUG:Importing EpsImagePlugin
06:17:56 DEBUG:Importing FitsImagePlugin
06:17:56 DEBUG:Importing FliImagePlugin
06:17:56 DEBUG:Importing FpxImagePlugin
06:17:56 DEBUG:Image: failed to import FpxImagePlugin: No module named 'olefile'
06:17:56 DEBUG:Importing FtexImagePlugin
06:17:56 DEBUG:Importing GbrImagePlugin
06:17:56 DEBUG:Importing GifImagePlugin
06:17:56 DEBUG:Importing GribStubImagePlugin
06:17:56 DEBUG:Importing Hdf5StubImagePlugin
06:17:56 DEBUG:Importing IcnsImagePlugin
06:17:56 DEBUG:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fcd8623bcb0>
06:17:56 DEBUG:Importing IcoImagePl

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


06:17:57 DEBUG:Starting new HTTPS connection (1): huggingface.co:443


06:17:57 DEBUG:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fcd7d47a690>
06:17:57 DEBUG:send_request_headers.started request=<Request [b'GET']>
06:17:57 DEBUG:send_request_headers.complete
06:17:57 DEBUG:send_request_body.started request=<Request [b'GET']>
06:17:57 DEBUG:send_request_body.complete
06:17:57 DEBUG:receive_response_headers.started request=<Request [b'GET']>
06:17:57 DEBUG:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 05 Mar 2025 14:17:57 GMT'), (b'Content-Type', b'application/json'), (b'Content-Length', b'21'), (b'Connection', b'keep-alive'), (b'Server', b'nginx/1.18.0'), (b'Access-Control-Allow-Origin', b'*')])
06:17:57 INFO:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
06:17:57 DEBUG:receive_response_body.started request=<Request [b'GET']>
06:17:57 DEBUG:receive_response_body.complete
06:17:57 DEBUG:response_closed.started
06:17:57 DEBUG:response_closed.complete
06: